# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.232325313423                   -0.50    6.0
  2   -7.249198740572       -1.77       -1.39    1.0
  3   -7.251240608763       -2.69       -1.84    4.0
  4   -7.251226851515   +   -4.86       -2.11    4.0
  5   -7.251270207457       -4.36       -2.25    3.0
  6   -7.251333665690       -4.20       -2.77    2.0
  7   -7.251338683561       -5.30       -3.89    2.0
  8   -7.251338763200       -7.10       -3.77    4.0
  9   -7.251338793732       -7.52       -4.16    3.0
 10   -7.251338795642       -8.72       -4.29    1.0
 11   -7.251338798666       -8.52       -5.25    3.0
 12   -7.251338798685      -10.72       -5.36    1.0
 13   -7.251338798692      -11.14       -5.48    1.0
 14   -7.251338798704      -10.91       -6.49    3.0
 15   -7.251338798705      -12.80       -6.76    3.0
 16   -7.251338798705      -13.97       -7.05    1.0
 17   -7.251338798705      -14.35       -7.56 

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.08640012311158889
[ Info: Arnoldi iteration step 2: normres = 0.6144850453735572
[ Info: Arnoldi iteration step 3: normres = 0.7053939608263041
[ Info: Arnoldi iteration step 4: normres = 0.2611774307933459
[ Info: Arnoldi iteration step 5: normres = 0.5432762078882754
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (2.16e-02, 8.02e-02, 4.34e-01, 3.16e-01, 1.16e-02)
[ Info: Arnoldi iteration step 6: normres = 0.2627841614153662
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (4.90e-03, 1.49e-01, 1.72e-01, 8.61e-02, 6.77e-02)
[ Info: Arnoldi iteration step 7: normres = 0.06746172124292028
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.57e-04, 9.22e-03, 8.78e-03, 2.64e-02, 5.60e-02)
[ Info: Arnoldi iteration step 8: normres = 0.1128043449792149
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (7.63e-06